In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import sqlite3

PATH = 'chromedriver'

In [2]:
driver = webdriver.Chrome(PATH)

# Scraping Function

In [3]:
#takes a link, the table id, and the index of the header row, returns the data as a df
def get_data(url, findId, headerIndex):
    driver.get(url)
    res = driver.find_element(By.ID, findId)
    rows = res.find_elements(By.TAG_NAME, "tr")
    #get column names from first row
    cols = []
    for ele in rows[headerIndex].find_elements(By.TAG_NAME, "th"):
        cols.append(ele.text)
    cols[1:]
    #create dataframe and append table rows to it
    stats = pd.DataFrame(columns = cols[1:])
    for row in rows:
        player = []
        for td in row.find_elements(By.TAG_NAME, "td"):
            player.append(td.text)
        #try except is needed here so that header rows don't throw an error when added to the df and are instead skipped
        try:
            stats.loc[len(stats)] = player
        except:
            continue
    return(stats)

In [4]:
#Get Player data for all major stats
player_wages = get_data('https://fbref.com/en/comps/22/wages/Major-League-Soccer-Wages','player_wages', 0)
gsc_stats  = get_data('https://fbref.com/en/comps/22/gca/Major-League-Soccer-Stats', 'stats_gca', 1)
defense_stats = get_data('https://fbref.com/en/comps/22/defense/Major-League-Soccer-Stats', 'stats_defense', 1)
keeper_stats = get_data('https://fbref.com/en/comps/22/keepers/Major-League-Soccer-Stats', 'stats_keeper', 1)
misc_stats = get_data('https://fbref.com/en/comps/22/misc/Major-League-Soccer-Stats', 'stats_misc', 1)
pass_stats = get_data('https://fbref.com/en/comps/22/passing/Major-League-Soccer-Stats', 'stats_passing', 1)
possession_stats = get_data('https://fbref.com/en/comps/22/possession/Major-League-Soccer-Stats', 'stats_possession', 1)
shooting_stats = get_data('https://fbref.com/en/comps/22/shooting/Major-League-Soccer-Stats', 'stats_shooting', 1)
time_stats = get_data('https://fbref.com/en/comps/22/playingtime/Major-League-Soccer-Stats', 'stats_playing_time', 1)



In [5]:
#Get Team data for all major stats
east_squads = get_data('https://fbref.com/en/comps/22/Major-League-Soccer-Stats#all_stats_squads_standard', 'results2022221Eastern-Conference_overall', 0)
west_squads = get_data('https://fbref.com/en/comps/22/Major-League-Soccer-Stats#all_stats_squads_standard', 'results2022221Western-Conference_overall', 0)

squad_goalkeeping = get_data('https://fbref.com/en/comps/22/keepers/Major-League-Soccer-Stats', 'stats_squads_keeper_for', 1)
squad_gsc  = get_data('https://fbref.com/en/comps/22/gca/Major-League-Soccer-Stats', 'stats_squads_gca_for', 1)
squad_defense = get_data('https://fbref.com/en/comps/22/defense/Major-League-Soccer-Stats', 'stats_squads_defense_for', 1)
squad_misc = get_data('https://fbref.com/en/comps/22/misc/Major-League-Soccer-Stats', 'stats_squads_misc_for', 1)
squad_pass = get_data('https://fbref.com/en/comps/22/passing/Major-League-Soccer-Stats', 'stats_squads_passing_for', 1)
squad_possession = get_data('https://fbref.com/en/comps/22/possession/Major-League-Soccer-Stats', 'stats_squads_possession_for', 1)
squad_shooting = get_data('https://fbref.com/en/comps/22/shooting/Major-League-Soccer-Stats', 'stats_squads_shooting_for', 1)
squad_time = get_data('https://fbref.com/en/comps/22/playingtime/Major-League-Soccer-Stats', 'stats_squads_playing_time_for', 1)


In [6]:
#creates function to add squad name to each squad df instead of an id number, since there will never be duplicate team names
squad_dict = {0:'Atlanta Utd', 1: 'Austin FC', 2:'CF Montréal', 3:'Charlotte FC', 4: 'Chicago Fire', 5:'Colorado Rapids', 6:'Columbus Crew', 7:'D.C. United', 8:'FC Cincinnati', 9:'FC Dallas', 10:'Houston Dynamo', 11:'Inter Miami', 12:'LA Galaxy', 13:'Los Angeles FC', 14:'Minnesota Utd', 15:'Nashville', 16:'New England', 17:'NY Red Bulls', 18:'NYCFC', 19:'Orlando City', 20:'Philadelphia', 21:'Portland Timbers', 22:'Real Salt Lake', 23:'San Jose', 24:'Seattle', 25:'Sporting KC', 26:'Toronto FC', 27:'Vancouver'}

def add_squads(df):
    df.reset_index(inplace = True)
    df['squad'] = df['index'].apply(lambda x: squad_dict[x])
    df.drop('index', inplace = True, axis = 1)

In [7]:
add_squads(squad_goalkeeping)
add_squads(squad_gsc)
add_squads(squad_defense)
add_squads(squad_misc)
add_squads(squad_pass)
add_squads(squad_possession)
add_squads(squad_shooting)
add_squads(squad_time)

In [8]:
#Combine east and west team dfs to one big team df
squads = pd.concat([east_squads, west_squads]).reset_index(drop=True)

In [9]:
squads.head()

,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes
0,Philadelphia,34,19,10,5,72,26,+46,67,1.97,59.4,37.9,+21.5,+0.63,"18,126",Dániel Gazdag - 22,Andre Blake,→ Conference Semifinals
1,CF Montréal,34,20,5,9,63,50,+13,65,1.91,55.3,37.0,+18.3,+0.54,"15,769",Romell Quioto - 15,Sebastian Breza,→ First Round
2,NYCFC,34,16,7,11,57,41,+16,55,1.62,58.4,38.2,+20.2,+0.59,"17,180",Valentín Castellanos - 13,Sean Johnson,→ First Round
3,NY Red Bulls,34,15,8,11,50,41,+9,53,1.56,45.5,39.2,+6.3,+0.18,"17,002",Lewis Morgan - 14,Carlos Coronel,→ First Round
4,FC Cincinnati,34,12,13,9,64,56,+8,49,1.44,55.6,48.1,+7.5,+0.22,"22,487","Brenner, Brandon Vazquez - 18",Roman Celentano,→ First Round


# Player Database prep and creation

In [29]:
defense_stats.reset_index()

#Create dictionary of player ids using name and squad as a composite so id can be used as a unique identifier for players, as 2 players could have the same name
player_ids = dict(zip(time_stats.Player+time_stats.Squad, time_stats.index))


In [30]:
#add loaner players that appear in wages but not other tables to player ids dict
index = list(player_ids.values())[-1]
player_list = player_wages['Player'] + player_wages['Squad']
for player in player_list:
    if player not in player_ids.keys():
        index += 1
        player_ids[player] = index

In [20]:
time_stats[time_stats['Player'] == 'Alan Franco']

,Player,Nation,Pos,Squad,Age,Born,MP,Min,Mn/MP,Min%,...,onGA,+/-,+/-90,On-Off,onxG,onxGA,xG+/-,xG+/-90,On-Off,Matches
271,Alan Franco,ar ARG,DF,Atlanta Utd,26-016,1996,31,"2,714",88,88.7,...,51,-9,-0.30,-1.08,48.7,44.8,+3.9,+0.13,-0.33,Matches
272,Alan Franco,ec ECU,MF,Charlotte,24-067,1998,10,782,78,25.6,...,13,-7,-0.81,-0.77,8.5,13.8,-5.3,-0.61,-0.69,Matches


In [31]:
def set_id(df):
    match_series = df['Player'] + df['Squad']
    df['player_id'] = match_series.map(lambda x: player_ids[x])
    df.drop('Player', axis=1, inplace = True)



In [34]:
#Add Player ID to each table instead of player name
set_id(gsc_stats)
set_id(defense_stats)
set_id(keeper_stats)
set_id(pass_stats)
set_id(possession_stats)
set_id(shooting_stats)
set_id(misc_stats)
set_id(player_wages)
time_stats['player_id'] = (time_stats['Player']+time_stats['Squad']).apply(lambda x: player_ids[x])
players = time_stats[['player_id', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s']]
time_stats.drop('Player', axis=1, inplace = True)

In [36]:
#Remove calculated and redundant fields from dataframes
player_wages.drop(['Nation', 'Pos', 'Squad', 'Age', 'Weekly Wages', 'Notes'], axis = 1, inplace = True)
gsc_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Matches', 'SCA', 'SCA90', 'GCA', 'GCA90'], axis =1, inplace = True)
defense_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Tkl%', 'Matches', 'Tkl+Int'], axis=1, inplace = True)
keeper_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Min', '90s', 'GA90', 'Save%', 'CS%', 'Matches', 'W', 'D', 'L'], axis=1, inplace = True)
pass_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Cmp%', 'Cmp%', 'Cmp%', 'Cmp%', 'A-xAG', '1/3', 'Matches'], axis=1, inplace = True)
possession_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Touches', 'Succ%', 'Matches'], axis=1, inplace = True)
shooting_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'npxG/Sh', 'G-xG', 'np:G-xG', 'Matches'], axis=1, inplace = True)
misc_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Int', 'TklW', 'Won%', 'Matches'], axis=1, inplace = True)
time_stats.drop(['Nation', 'Pos', 'Squad', 'Age', 'Born', 'Mn/MP', 'Min%', '90s', '+/-', '+/-90', 'xG+/-', 'xG+/-90', 'Matches'], axis=1, inplace = True)

In [37]:
# Rename columns for database upload, drop calculated columns that weren't dropped earlier due to column name overlap
keeper_stats.rename(columns = {'Starts':'starts', 'GA':'ga', 'SoTA':'sota', 'Saves':'saves', 'CS':'cs', 'PKatt':'pkAtt', 'PKA':'pka', 'PKsv':'pksv', 'PKm':'pkm', 'player_id':'player_id'}, inplace = True)
gsc_stats.set_axis(['scPassLive', 'scPassDead', 'scDrib', 'scSh', 'scFld', 'scDef', 'gcPassLive', 'gcPassDead', 'gcDrib', 'gcSh', 'gcFld', 'gcDef', 'player_id' ], axis=1, inplace=True)
defense_stats.set_axis(['tkl', 'tklW', 'tklDefThird', 'tklMidThird', 'tklAttThird', 'tklVsDrib', 'tklAttVsDrib', 'past','blk', 'blkSh', 'blkPass', 'int', 'clr', 'err', 'player_id'], axis=1, inplace = True)
pass_stats.set_axis(['totCmp', 'totAtt', 'totDist', 'prgDist', 'shortCmp', 'shortAtt', 'medCmp', 'medAtt', 'lngCmp', 'lngAtt', 'ast', 'xAG', 'xA', 'kp', 'ppa', 'crsPa', 'progPasses', 'player_id' ], axis=1, inplace=True)
pass_stats.drop(['totCmp', 'totAtt'], axis = 1, inplace = True)
possession_stats.set_axis(['defPen', 'defThird', 'midThird', 'attThird', 'attPen', 'liveTouches', 'succDrib', 'attDrib', 'mis', 'tackled', 'passRec', 'progPassRec', 'player_id'], axis=1, inplace=True)
shooting_stats.set_axis(['gls', 'sh', 'sot', 'avgDist', 'fk', 'pk', 'pkAtt', 'xG', 'npxG', 'player_id'], axis=1, inplace=True)
misc_stats.set_axis(['crdY', 'crdR', 'crdY2', 'fls', 'fld', 'off', 'crs', 'pkWon', 'PKcon', 'og', 'recov', 'airW', 'airL', 'player_id'], axis=1, inplace=True)
time_stats.set_axis(['mp', 'min', 'starts', 'mn/start', 'compl', 'subs', 'mn/sub', 'unSub', 'ppm', 'onG', 'onGa', 'on-off', 'onxG', 'onxGa', 'xon-off', 'player_id'], axis=1, inplace=True)
players.set_axis(['id', 'name', 'nation', 'pos', 'squad', 'age', 'born', '90s'], axis = 1, inplace = True)
player_wages.set_axis(['wage', 'player_id'], axis = 1, inplace = True)

In [38]:
#remove days from age, make it an int field
players['age'] = players['age'].apply(lambda x: x[0:2])
avg_age = int(players[players['age'] != ''].age.astype('int').mean())
players['age'] = players['age'].apply(lambda x: avg_age if x == '' else x)
players['age'] = players['age'].astype('int')

C:\Users\Brian\AppData\Local\Temp/ipykernel_18796/2856548900.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players['age'] = players['age'].apply(lambda x: x[0:2])
C:\Users\Brian\AppData\Local\Temp/ipykernel_18796/2856548900.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players['age'] = players['age'].apply(lambda x: avg_age if x == '' else x)
C:\Users\Brian\AppData\Local\Temp/ipykernel_18796/2856548900.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [39]:
#For players without an age impute the avg age
avg_born = players[players['born'] != ''].born.astype('int').mean()
players['born'] = players['born'].apply(lambda x: avg_born if x == '' else x)
players['born'] = players['born'].astype('int')

C:\Users\Brian\AppData\Local\Temp/ipykernel_18796/983669323.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players['born'] = players['born'].apply(lambda x: avg_born if x == '' else x)
C:\Users\Brian\AppData\Local\Temp/ipykernel_18796/983669323.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players['born'] = players['born'].astype('int')


In [40]:
#convert on-offs to floats
time_stats['on-off'] = time_stats['on-off'].apply(lambda x: 0 if x == '' else x)
time_stats['on-off'] = time_stats['on-off'].astype('float')

time_stats['xon-off'] = time_stats['xon-off'].apply(lambda x: 0 if x == '' else x)
time_stats['xon-off'] = time_stats['xon-off'].astype('float')

time_stats.head()

,mp,min,starts,mn/start,compl,subs,mn/sub,unSub,ppm,onG,onGa,on-off,onxG,onxGa,xon-off,player_id
0,23,432,2,69,0,21,14,7,2.13,8,5,-0.85,8.3,7.6,-0.56,0
1,32,"2,522",29,85,24,3,19,0,1.25,36,41,0.83,41.9,38.2,0.98,1
2,31,"2,264",27,79,11,4,33,1,1.82,38,28,-0.28,40.7,24.0,0.26,2
3,15,382,0,,0,15,25,14,0.67,7,6,0.84,6.2,5.9,0.63,3
4,26,"1,898",22,83,12,4,20,4,1.46,32,33,0.73,30.8,29.7,0.24,4


In [44]:
#Convert Annual Wages to numeric dollar value
import re
def stripCommas(val):
    return int(re.sub( ',' ,'',val.split(' ')[0][1:]))
    
player_wages['wage'] = player_wages['wage'].apply(stripCommas)

In [45]:
#add to db next
#connect to mlsdb and create/update players table
with sqlite3.connect('mlsdb.sqlite') as conn:
    players.to_sql('players', conn, if_exists='replace', index = False, dtype = {'id':'INTEGER', 'name':'TEXT', 'nation':'TEXT', 'pos':'TEXT', 'squad':'TEXT', 'age':'INTEGER', 'born':'INTEGER', '90s':'REAL'})
    keeper_stats.to_sql('playerKeepers', conn, if_exists='replace', index = False, dtype={'starts':'INTEGER', 'ga':'INTEGER', 'sota':'INTEGER', 'saves':'INTEGER', 'cs':'INTEGER', 'ppkAtt':'INTEGER', 'pka':'INTEGER', 'pksv':'INTEGER', 'pkm':'INTEGER', 'player_id':'INTEGER'})
    defense_stats.to_sql('playerDefense', conn, if_exists='replace', index = False, dtype={'tkl':'INTEGER', 'tklW':'INTEGER', 'tklDefThird':'INTEGER', 'tklMidThird':'INTEGER', 'tklAttThird':'INTEGER', 'tklVsDrib':'INTEGER', 'tklAttVsDrib':'INTEGER', 'past':'INTEGER', 'blk':'INTEGER', 'blkSh':'INTEGER', 'blkPass':'INTEGER', 'int':'INTEGER', 'clr':'INTEGER', 'err':'INTEGER', 'player_id':'INTEGER'}) 
    gsc_stats.to_sql('playerGsc', conn, if_exists='replace', index = False, dtype = {'scPassLive':'INTEGER', 'scPassDead':'INTEGER', 'scDrib':'INTEGER', 'scSh':'INTEGER', 'scFld':'INTEGER', 'scDef':'INTEGER','gcPassLive':'INTEGER', 'gcPassDead':'INTEGER', 'gcDrib':'INTEGER', 'gcSh':'INTEGER', 'gcFld':'INTEGER', 'gcDef':'INTEGER', 'player_id':'INTEGER'})
    misc_stats.to_sql('playerMisc', conn, if_exists='replace', index = False, dtype={'crdY':'INTEGER', 'crdR':'INTEGER', 'crdY2':'INTEGER', 'fls':'INTEGER', 'fld':'INTEGER', 'off':'INTEGER', 'crs':'INTEGER', 'pkWon':'INTEGER', 'PKcon':'INTEGER','og':'INTEGER', 'recov':'INTEGER', 'airW':'INTEGER', 'airL':'INTEGER', 'player_id':'INTEGER'})  
    pass_stats.to_sql('playerPassing', conn, if_exists='replace', index = False, dtype={'totDist':'INTEGER', 'prgDist':'INTEGER', 'shortCmp':'INTEGER', 'shortAtt':'INTEGER', 'medCmp':'INTEGER', 'medAtt':'INTEGER','lngCmp':'INTEGER', 'lngAtt':'INTEGER', 'ast':'INTEGER', 'xAG': 'Integer', 'xA':'INTEGER', 'kp':'INTEGER', 'ppa':'INTEGER', 'crsPa':'INTEGER', 'progPasses':'INTEGER','player_id':'INTEGER'})
    possession_stats.to_sql('playerPossession', conn, if_exists='replace', index = False, dtype={'defPen':'INTEGER', 'defThird':'INTEGER', 'midThird':'INTEGER', 'attThird':'INTEGER', 'attPen':'INTEGER', 'liveTouches':'INTEGER', 'succDrib':'INTEGER', 'attDrib':'INTEGER', 'mis':'INTEGER', 'tackled':'INTEGER', 'passRec':'INTEGER', 'progPassRec':'INTEGER', 'player_id':'INTEGER'})
    shooting_stats.to_sql('playerShooting', conn, if_exists='replace', index = False, dtype={'gls':'INTEGER', 'sh':'INTEGER', 'sot':'INTEGER', 'avgDist':'INTEGER', 'fk':'INTEGER', 'pk':'INTEGER', 'pkAtt':'INTEGER', 'xG':'INTEGER', 'npxG':'INTEGER', 'player_id':'INTEGER'})    
    time_stats.to_sql('playerTime', conn, if_exists='replace', index = False, dtype = {'mp':'INTEGER', 'min':'INTEGER', 'starts':'INTEGER', 'mn/start':'INTEGER', 'compl':'INTEGER', 'subs':'INTEGER', 'mn/sub':'INTEGER', 'unSub':'INTEGER','ppm':'REAL', 'onG':'INTEGER', 'onGa':'INTEGER', 'on-off':'REAL', 'onxG':'REAL', 'onxGa':'REAL', 'xon-off':'REAL', 'player_id':'INTEGER'}) 
    player_wages.to_sql('playerWages', conn, if_exists='replace', index = False, dtype={'wage': 'INTEGER', 'player_id':'INTEGER'})

# Squad Database Prep and Creation

In [46]:
#we don't need to set ID here as squad name is an adaquate primary key since it will never be duplicated
squads.drop(['Pts/MP', 'Pts', 'GD', 'GF', 'xGD/90', 'xGD', 'Top Team Scorer', 'Notes', 'xG'], axis=1, inplace = True)
squad_goalkeeping.drop(['# Pl', 'MP', 'Starts', 'Min', '90s', 'GA', 'GA90', 'Save%', 'W', 'D', 'L', 'CS%'], axis = 1, inplace = True)
squad_gsc.drop(['# Pl', '90s', 'SCA90', 'GCA90', 'SCA', 'GCA'], axis = 1, inplace = True)
#drop first tkl after renaming
squad_defense.drop(['# Pl', '90s', 'Tkl%', 'Tkl+Int'], axis = 1, inplace = True)
squad_misc.drop(['# Pl', '90s', 'Int', 'TklW', 'Won%'], axis = 1, inplace = True)
#drop first cmp after renaming
squad_pass.drop(['# Pl', '90s', 'Cmp%', 'A-xAG', '1/3'], axis = 1, inplace = True)
#drop first # pl after renaming
squad_possession.drop([ '90s', 'Touches', 'Succ%'], axis = 1, inplace = True)
squad_shooting.drop(['# Pl', '90s', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'npxG/Sh', 'G-xG', 'np:G-xG'], axis = 1, inplace = True)
squad_time.drop(['# Pl', 'MP', 'Mn/MP', 'Min%', '90s', 'Starts', 'Mn/Start', 'Compl', 'PPM', '+/-', '+/-90', 'xG+/-', 'xG+/-90', 'onG', 'onGA', 'onxG', 'onxGA'], axis = 1, inplace = True)

In [47]:
squads.head(1)

,Squad,MP,W,D,L,GA,xGA,Attendance,Goalkeeper
0,Philadelphia,34,19,10,5,26,37.9,"18,126",Andre Blake


In [48]:
squads.set_axis(['squad', 'mp', 'w', 'd', 'l', 'ga', 'xGA', 'attendence', 'goalkeeper'], axis=1, inplace=True)
squad_goalkeeping.set_axis(['sota', 'saves', 'cs', 'pkAtt', 'pkA', 'pkSv', 'pkM', 'squad'], axis=1, inplace=True)
squad_defense.set_axis(['tkl', 'tklW', 'tklDefThird', 'tklMidThird', 'tklAttThird', 'tklVsDrib', 'attVsDrib', 'past', 'blk', 'blkSh', 'blkPass', 'int', 'clr', 'err', 'squad'], axis=1, inplace=True)
squad_misc.set_axis(['crdY', 'crdR', 'crdY2', 'fls', 'fld', 'off', 'crs', 'pkWon', 'pkCon', 'og', 'recov', 'airW', 'airL', 'squad'], axis=1, inplace=True)
squad_pass.set_axis(['totCmp', 'totAtt', 'totDist', 'prgDist', 'shortCmp', 'shortAtt', 'medCmp', 'medAtt', 'lngCmp', 'lngAtt', 'ast', 'xAG', 'xA', 'kp', 'ppa', 'crsPa', 'progPasses', 'squad'], axis=1, inplace=True)
squad_possession.set_axis(['# Pl', 'poss', 'defPen', 'defThird', 'midThird', 'attThird', 'attPen', 'live', 'dribW', 'dribAtt', 'mis', 'dis', 'rec', 'progRec', 'squad' ], axis=1, inplace=True)
squad_possession.drop('# Pl', axis = 1, inplace = True)
squad_shooting.set_axis(['gls', 'sh', 'sot', 'avgDist', 'fk', 'pkW', 'pkAtt', 'xG', 'npxG', 'squad'], axis=1, inplace=True)
squad_time.set_axis(['age','min','subs','mn/sub','unsub', 'squad'], axis=1, inplace=True)
squad_gsc.set_axis(['scPassLive', 'scPassDead', 'scDrib', 'scSh', 'scFld', 'scDef', 'gcPassLive', 'gcPassDead', 'gcDrib', 'gcSh', 'gcFld', 'gcDef', 'squad' ], axis=1, inplace=True)


In [49]:
#add to db next
#connect to mlsdb and create/update players table
with sqlite3.connect('mlsdb.sqlite') as conn:
    squads.to_sql('squads', conn, if_exists='replace', index = False, dtype={'squad':'TEXT', 'mp':'INTEGER', 'w':'INTEGER', 'd':'INTEGER', 'l':'INTEGER', 'ga':'INTEGER', 'xGA':'REAL', 'attendence':'INTEGER', 'goalkeeper':'TEXT'})
    squad_goalkeeping.to_sql('squadKeepers', conn, if_exists='replace', index = False, dtype={'sota':'INTEGER', 'saves':'INTEGER', 'cs':'INTEGER', 'pkAtt':'INTEGER', 'pkA':'INTEGER', 'pkSv':'INTEGER', 'pkM':'INTEGER', 'squad':'TEXT'})
    squad_defense.to_sql('squadDefense', conn, if_exists='replace', index = False, dtype= {'tkl':'INTEGER', 'tklW':'INTEGER', 'tklDefThird':'INTEGER', 'tklMidThird':'INTEGER', 'tklAttThird':'INTEGER', 'tklVsDrib':'INTEGER', 'attVsDrib':'INTEGER', 'past':'INTEGER', 'blk':'INTEGER', 'blkSh':'INTEGER', 'blkPass':'INTEGER', 'int':'INTEGER', 'clr':'INTEGER', 'err':'INTEGER', 'squad':'TEXT'}) 
    squad_gsc.to_sql('squadGsc', conn, if_exists='replace', index = False, dtype = {'scPassLive':'INTEGER', 'scPassDead':'INTEGER', 'scDrib':'INTEGER', 'scSh':'INTEGER', 'scFld':'INTEGER', 'scDef':'INTEGER','gcPassLive':'INTEGER', 'gcPassDead':'INTEGER', 'gcDrib':'INTEGER', 'gcSh':'INTEGER', 'gcFld':'INTEGER', 'gcDef':'INTEGER', 'squad':'TEXT'})
    squad_misc.to_sql('squadMisc', conn, if_exists='replace', index = False, dtype={'crdY':'INTEGER', 'crdR':'INTEGER', 'crdY2':'INTEGER', 'fls':'INTEGER', 'fld':'INTEGER', 'off':'INTEGER', 'crs':'INTEGER', 'pkWon':'INTEGER', 'pkCon':'INTEGER', 'og':'INTEGER', 'recov':'INTEGER', 'airW':'INTEGER', 'airL':'INTEGER', 'squad':'REAL'})  
    squad_pass.to_sql('squadPassing', conn, if_exists='replace', index = False, dtype = {'totCmp':'INTEGER', 'totAtt':'INTEGER', 'totDist':'INTEGER', 'prgDist':'INTEGER', 'shortCmp':'INTEGER', 'shortAtt':'INTEGER', 'medCmp':'INTEGER', 'medAtt':'INTEGER', 'lngCmp':'INTEGER', 'lngAtt':'INTEGER', 'ast':'INTEGER', 'xAG':'INTEGER', 'xA':'INTEGER', 'kp':'INTEGER', 'ppa':'INTEGER', 'crsPa':'INTEGER', 'progPasses':'INTEGER', 'squad':'TEXT'})
    squad_possession.to_sql('squadPossession', conn, if_exists='replace', index = False, dtype={'poss':'INTEGER', 'defPen':'INTEGER', 'defThird':'INTEGER', 'midThird':'INTEGER', 'attThird':'INTEGER', 'attPen':'INTEGER', 'live':'INTEGER', 'dribW':'INTEGER', 'dribAtt':'INTEGER', 'mis':'INTEGER', 'dis':'INTEGER', 'rec':'INTEGER', 'progRec':'INTEGER','squad':'TEXT'})
    squad_shooting.to_sql('squadShooting', conn, if_exists='replace', index = False, dtype={'gls':'INTEGER', 'sh':'INTEGER', 'sot':'INTEGER', 'avgDist':'REAL', 'fk':'INTEGER', 'pkW':'INTEGER', 'pkAtt':'INTEGER', 'xG':'REAL', 'npxG':'REAL', 'squad':'TEXT'})    
    squad_time.to_sql('squadTime', conn, if_exists='replace', index = False, dtype={'age':'REAL', 'min':'INTEGER', 'subs':'INTEGER', 'mn/sub':'INTEGER', 'unsub':'INTEGER', 'squad':'TEXT'}) 

,id,name,nation,pos,squad,age,born,90s
0,0,Paxten Aaronson,us USA,"MF,FW",Philadelphia,19,2003,4.8
1,1,Lalas Abubakar,gh GHA,DF,Colorado Rapids,27,1994,28.0
2,2,Nicolás Acevedo,uy URU,"MF,DF",NYCFC,23,1999,25.2
3,3,Ifunanyachi Achara,us USA,"MF,FW",Toronto FC,25,1997,4.2
4,4,Bryan Acosta,hn HON,MF,Colorado Rapids,28,1993,21.1
...,...,...,...,...,...,...,...,...
867,867,Rida Zouhir,ca CAN,MF,CF Montréal,18,2003,1.4
868,868,Ethan Zubak,us USA,"FW,MF",Nashville,24,1998,2.3
869,869,Dario Župarić,ba BIH,DF,Portland Timbers,30,1992,24.4
870,870,Graham Zusi,us USA,DF,Sporting KC,36,1986,18.4


,Annual Wages,player_id
0,14000000,374
1,7350000,742
2,6000000,351
3,5100000,355
4,4750000,354
...,...,...
793,65500,902
794,65500,903
795,65500,601
796,65500,426
